# **Part B**

In [14]:
#import the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score

In [15]:
#dataset
(X_train,Y_train),(X_test,Y_test) = tf.keras.datasets.cifar10.load_data()
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,test_size=0.3)

In [16]:
np.bincount(Y_train.T[0])

array([3465, 3453, 3486, 3497, 3559, 3504, 3493, 3517, 3541, 3485])

In [17]:
encoder = OneHotEncoder(sparse_output=False)
Y_train = pd.DataFrame(Y_train)
Y_train.columns = ['Type']
one_hot_encoded = encoder.fit_transform(Y_train[['Type']])
y_train_transformed = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Type']))
y_train_transformed.shape

(35000, 10)

In [18]:
encoder = OneHotEncoder(sparse_output=False)
Y_test = pd.DataFrame(Y_test)
Y_test.columns = ['Type']
one_hot_encoded = encoder.fit_transform(Y_test[['Type']])
y_test_transformed = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Type']))
y_test_transformed.shape

(15000, 10)

You have learnt about data augmentation in the theoretical assignment, Now you need to use it in the above dataset (You can do it using ImageDataGenerator

In [19]:
# Create an ImageDataGenerator object
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./32)

train = datagen.flow(X_train, y_train_transformed, batch_size=32)


In [20]:
print(train)

In [21]:
X_train.shape

(35000, 32, 32, 3)

Use Visualization libraries to see how data augmentation works.

Train a CNN model of your choice and evaluate on the test data using weighted F1 scores ( Do not just try with a single model use multiple models, for example- one with transfer learning or with a different model architecture) at the end compare the performance of the models, and write what do you infer from the same

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [24]:
model.fit(train, epochs=10)

Epoch 1/10
1094/1094 [==============================] - 17s 15ms/step - loss: 1.5454 - accuracy: 0.4527
Epoch 2/10
1094/1094 [==============================] - 16s 15ms/step - loss: 1.1242 - accuracy: 0.6077
Epoch 3/10
1094/1094 [==============================] - 16s 15ms/step - loss: 0.9467 - accuracy: 0.6768
Epoch 4/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.8216 - accuracy: 0.7242
Epoch 5/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.7341 - accuracy: 0.7541
Epoch 6/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.6661 - accuracy: 0.7812
Epoch 7/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.6108 - accuracy: 0.7994
Epoch 8/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.5834 - accuracy: 0.8098
Epoch 9/10
1094/1094 [==============================] - 17s 15ms/step - loss: 0.5693 - accuracy: 0.8186
Epoch 10/10
1094/1094 [==============================] - 17s 15m

In [25]:
model.evaluate(X_test, y_test_transformed)

469/469 [==============================] - 3s 5ms/step - loss: 36.7050 - accuracy: 0.6259


[36.704994201660156, 0.6259333491325378]

In [26]:
predictions = np.argmax(model.predict(X_test), axis=1)

encoder = OneHotEncoder(sparse_output=False)
predictions = pd.DataFrame(predictions)
predictions.columns = ['Type']
one_hot_encoded = encoder.fit_transform(predictions[['Type']])
predictions_transformed = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Type']))

f1_score(y_test_transformed, predictions_transformed, average='weighted')

469/469 [==============================] - 2s 5ms/step


0.6192812935597565